# Model 10: Denoising Autoencoder

Play with Gaussian Noise and Dropout in autoencoder.  Taking a brief break from xVertSeg to explore some ideas at our final presentation.



## Imports and Constants, etc.

In [ ]:
import datetime
import importlib
import keras
from keras.layers import (Dense, SimpleRNN, Input, Conv1D, 
                          LSTM, GRU, AveragePooling3D, MaxPooling3D, GlobalMaxPooling3D,
                          Conv3D, UpSampling3D, BatchNormalization, Concatenate, Add,
                          GaussianNoise, Dropout
                         )
from keras.models import Model
import nibabel as nib
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import projd
import random
import re
import scipy
import shutil
import SimpleITK # xvertseg MetaImage files
import sys
from sklearn.model_selection import train_test_split
import uuid

import matplotlib.pyplot as plt # data viz
import seaborn as sns # data viz

import imageio # display animated volumes
from IPython.display import Image # display animated volumes

from IPython.display import SVG # visualize model
from keras.utils.vis_utils import model_to_dot # visualize model

# for importing local code
src_dir = str(Path(projd.cwd_token_dir('notebooks')) / 'src') # $PROJECT_ROOT/src
if src_dir not in sys.path:
    sys.path.append(src_dir)

import util
import preprocessing
import datagen
import modelutil
import xvertseg


MODEL_NAME = 'model_10'
SEED = 25 # random seed
EPOCHS = 100
BATCH_SIZE = 4
PATCH_SHAPE = (32, 32, 32)
VALIDATION_SPLIT = 0.25

DATA_DIR = Path('/data2').expanduser()
# UVMMC
NORMAL_SCANS_DIR = DATA_DIR / 'uvmmc/nifti_normals'
PROJECT_DATA_DIR = DATA_DIR / 'uvm_deep_learning_project'
PP_IMG_DIR = PROJECT_DATA_DIR / 'uvmmc' / 'preprocessed' # preprocessed scans dir
PP_MD_PATH = PROJECT_DATA_DIR / 'uvmmc' / 'preprocessed_metadata.pkl'
# xVertSeg
XVERTSEG_DIR = DATA_DIR / 'xVertSeg.v1'
PP_XVERTSEG_DIR = PROJECT_DATA_DIR / 'xVertSeg.v1' / 'preprocessed' # preprocessed scans dir
PP_XVERTSEG_MD_PATH = PROJECT_DATA_DIR / 'xVertSeg.v1' / 'preprocessed_metadata.pkl'


MODELS_DIR = PROJECT_DATA_DIR / 'models'
LOG_DIR = PROJECT_DATA_DIR / 'log'
TENSORBOARD_LOG_DIR = PROJECT_DATA_DIR / 'tensorboard' / MODEL_NAME
TMP_DIR = DATA_DIR / 'tmp'

for d in [DATA_DIR, NORMAL_SCANS_DIR, PROJECT_DATA_DIR, PP_IMG_DIR, MODELS_DIR, LOG_DIR, 
          TENSORBOARD_LOG_DIR, TMP_DIR, PP_MD_PATH.parent, PP_XVERTSEG_DIR, PP_XVERTSEG_MD_PATH.parent]:
    if not d.exists():
        d.mkdir(parents=True)
        
%matplotlib inline
sns.set()

# I love u autoreload!
%load_ext autoreload
%autoreload 2

## Data Generation

In [ ]:
train_gen, val_gen = datagen.get_nifti_datagens(
    preprocessed_metadata_path=PP_MD_PATH, batch_size=BATCH_SIZE, seed=SEED, validation_split=VALIDATION_SPLIT,
    crop_shape=PATCH_SHAPE)

## Build Model

Downsampled agressively with strided convolutions to fit in memory.  Residual blocks.  Global pooling at the end to classify.


In [ ]:

def downsampling_block(x, n_a, dropout=None):
    x = Dropout(rate=dropout)(x)
    x = MaxPooling3D(padding='same')(x)
    x = Conv3D(n_a, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(n_a, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    return x


def upsampling_block(x, n_a, dropout=None):
    x = Dropout(rate=dropout)(x)
    x = UpSampling3D()(x) # dropout dimming?
    x = Conv3D(n_a, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(n_a, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    return x

    
def residual_block(x, n_a, dropout=None):
    '''
    n_l: number of layers/convolutions in the residual path.
    '''
    x_i = x # initial x
    if dropout is not None:
        x = Dropout(rate=dropout)(x)
    x = Conv3D(n_a, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(n_a, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)        
    x = Add()([x_i, x])  
    return x

    
def build_model(input_shape, n_a=16, n_r=8, n_d=3, dropout=None, noise=None):
    '''
    3D convolutional encoder-decoder autoencoder.
    3 poolings reduce input from shape to 1/512th the original size.
    Channel depth increases to 8 * n_a
    a very respectable compression factor.
    
    dropout: proportion of activation of input to drop out. 0.0 to 1.0
    noise: std dev of noise added to input activation.
    returns: model
    '''

    x_input = Input(shape=input_shape)
    x = x_input
    
    # noise regularization
    if noise: 
        x = GaussianNoise(stddev=noise)(x)

    x = Conv3D(n_a, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)

    # dropout, maxpool, conv, conv
    for d in range(1, n_d+1):
        x = downsampling_block(x, n_a=n_a*(2**d), dropout=dropout)

    # residual: dropout, conv, conv
    for r in range(n_r):
        x = residual_block(x, n_a=n_a*(2**n_d), dropout=dropout)
        
    # dropout, upsampling, conv, conv
    for d in range(n_d, 0, -1):
        x = upsampling_block(x, n_a=n_a*(2**(d-1)), dropout=dropout)

    y = Conv3D(1, kernel_size=(1, 1, 1), activation='sigmoid')(x)
    
    model = Model(inputs=x_input, outputs=y)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

   
   

In [ ]:
n_a = 16 # base number of channels.
n_r = 8 # number of residual blocks
n_d = 3 # number of downsamplings.  aka depth.
dropout = 0.2 # dropout rate
noise = 0.003 # input gaussian noise std dev
model = build_model(input_shape=PATCH_SHAPE + (1,), n_a=12, n_r=4, n_d=3, dropout=dropout, noise=noise)


In [ ]:
print(model.summary())


In [ ]:
SVG(model_to_dot(model).create(prog='dot', format='svg'))


## Train and Evaluate Model

Training has callbacks to save model every epoch, to log performance stats every epoch to a csv file and tensorboard.



In [ ]:
history, log_path = modelutil.train_model(
    model, train_gen, val_gen, epochs=100, batch_size=BATCH_SIZE, models_dir=MODELS_DIR, model_name=MODEL_NAME, 
    log_dir=LOG_DIR, tensorboard_log_dir=TENSORBOARD_LOG_DIR, max_queue_size=20, use_multiprocessing=True)


## Visualize Training Progress

In [ ]:
# read metrics from the log file
log_path = sorted(LOG_DIR.glob(f'{MODEL_NAME}*_log.csv'), reverse=True)[0]
print('most recent log_path:', log_path)
metrics = pd.read_csv(log_path)

In [ ]:
print(pd.concat([metrics[::10], metrics[-1:]])) # every 10th metric and the last one

In [ ]:
# Plot Training and Validation Accuracy 
axes = plt.gca()
axes.set_ylim([0.0,1.0]) # Show results on 0..1 range
plt.plot(metrics["acc"])
plt.plot(metrics["val_acc"])
plt.legend(['Training Accuracy', "Validation Accuracy"])
plt.show()

# Plot Training and Validation Loss
plt.plot(metrics["loss"])
plt.plot(metrics["val_loss"])
plt.legend(['Training Loss', "Validation Loss"])
plt.show()



It is easier to see on tensorboard the validation loss is still dropping, slightly, maybe, compared to model 4, where validation loss is climbing.

## Show Effect of Autoencoder Training

Use models from different training epochs to encode images.


In [ ]:
for viz in modelutil.vizualize_predictions_by_epochs(MODELS_DIR, MODEL_NAME, [1, EPOCHS//10, EPOCHS], train_gen, val_gen, step=1):
    display(viz)
    
